# This notebook will be utilised to develop and implement the cointegration pairs strategy model.

Import necessary libraries/modules.

In [1]:
import numpy as np # This module is imported to allow for array computations and data manipulations.
import pandas as pd # Same as above.
from statsmodels.tsa.stattools import coint, adfuller  # This module is imported for the Engle-Granger cointegration test implementation.
import time # Imported to time code.
import os # imported to handle directories and pathnames.
import glob # same as os.
import vectorbt as vbt # This library will be utilised to perform the backtesting of the algo strategy across the historical data.
import statsmodels.api as sm # Used for regression fitting in the Engle-Granger cointegration test.
import itertools

Load the open-high-low-close-volume data generated in market_generator.py.

In [3]:
# Directory where the generated CSV files are stored.

### Make sure you use the correct path where the csv are stored !!!

csv_dir = '/Users/spyros_marke/Desktop/MCE_assessment'

# List of all CSV files in the directory
csv_files = glob.glob(os.path.join(csv_dir, '*.csv'))

# Storing data in a dictionary where the key is the trading-pair name
# and the value is the dataframe itself.

data_dict = {}

for file in csv_files:
    pair = os.path.basename(file).replace('_data.csv', '').replace('_', '/')
    df = pd.read_csv(file)
    data_dict[pair] = df

We have the required data for ~n markets (depending the number chosen in market_generator.py) in monthly availability and minute resolution.  Next steps are developing the cointegrated pairs strategy and backtest it across the monthly history of the trading pairs.

# An introduction to Cointegration.

The word "integration" within cointegration refers to an integrated time series of order d denoted by I(d). 
In general, price,rate and yield data can be assumed to be I(1) series while returns can be assumed to be I(0) series.
What is convenient about I(0) series is that they are $\textbf{weak-sense stationary} $. 
This implies that the mean and variance of the time series are finite and do not change over time. 
Effectively, if the time series wanders far from the mean, the time-invariant property will "drag" the series back to make sure that its mean will not change.

But returns are I(0) and we cannot trade returns.
What we can actually trade is the prices and they are non-stationary, so we can try and create stationarity by looking at pairs of assets.
According to Alexander (2002) $x_t$ and $y_t$ are cointegrated, if $x_t$ and $y_t$ are I(1) series such that:

$z_t = x_t -\beta*y_t$ is an I(0) series.

Cointegrations allows to construct a stationary time series from 2 asset prices, if only we can find the cointegration coefficient $\beta$. 
Then we can apply a mean-reversion strategy to trade both assets at the same time weighted by $\beta$. 
There is no guarantee that such $\beta$ always exists, and you should look for other asset pairs if no such $\beta$ can be found.

# How to find $\beta$?

The two workhorses of finding the cointegration coefficient $\beta$ 
(or cointegration vector when there are more than 2 assets) are the Engle-Granger test (Engle, 1987) and the Johansen test.

We will start woring with the Engle-Granger test as it is much simpler and works well for pairs of assets. 

The idea of Engle-Granger test is that we perform a linear regression between the two asset prices and check if the residual is stationary using the Augmented Dick-Fuller (ADF) test. If the residual is stationary, then the two asset prices are cointegrated. The cointegration coefficient is obtained as the coefficient of the regressor.

One problem that arises is which variable should be chosen as dependent and which as independent. To overcome this caveat, we run the linear regression twice using each asset as the dependent variable, respectively. 
The final $\beta$ would be the combination that yields a more significant ADF test.

We expect the Engle-Granger test to work in our case because we are comparing pairs of two assets every time. But what if we have more than two assets? If we still apply the abovementioned heuristic, we will have to run multiple linear regressions, which is rather cumbersome. This is where Johansen test could come in handy. So if the results are insufficient we will have to revert to the Johansen approach.

# Choose a number of trading pairs to perform cointegration for. 

In principle, you could use as many as you wanted generated from the .py file but the n choose 2 number
of combinations are going to be enough for smaller numbers of n.

In [16]:
pairs = list(data_dict.keys())[10:25]

In [4]:
def EG_cointegration(df1, df2):

    """
    Function that performs regression between the two asset prices and checks
    whether the residual is stationary utilising the ADF test. Note that the
    regression is run with both series being used as dependent and independent 
    series so we can get the beta that yields the most significant ADF.
    """
    
    df1_const = sm.add_constant(df1)  # Add a constant term
    model1 = sm.OLS(df2, df1_const).fit()
    residuals1 = model1.resid
    adf_result1 = adfuller(residuals1)
    
    df2_const = sm.add_constant(df2)  # Add a constant term
    model2 = sm.OLS(df1, df2_const).fit()
    residuals2 = model2.resid
    adf_result2 = adfuller(residuals2)
    
    if adf_result1[1] < adf_result2[1]:
        return adf_result1[1], model1.params.iloc[0],model1.params.iloc[1]  # Return p-value and beta
    else:
        return adf_result2[1], model2.params.iloc[0], model2.params.iloc[1]


The following cell produces a list with the cointegrated pairs of traidng pairs (assets). Their beta values will be used to calculate the spread to be utilised
for the mean reversion trading strategy.

In [5]:
EG_results = []

for i in range(len(pairs)):
    for j in range(i+1,len(pairs)):

        pair1 = data_dict[pairs[i]]['close']
        pair2 = data_dict[pairs[j]]['close']
        p_value,alpha,beta = EG_cointegration(pair1,pair2)

        if p_value < 0.05:
            # print(f'Null Hypothesis Rejected! Pairs {pairs[i]} and {pairs[j]} are cointegrated')
            EG_results.append((pairs[i],pairs[j],p_value,beta))
            
for result in EG_results:
    print(f"Pair: {result[0]} and {result[1]}, P-value: {result[2]}, Cointegration coeeficient beta: {result[3]}")

After identifying cointegrated trading pairs, we have the pair names and the beta coefficients. We can then straightforwardly calculate 
the array of their spreads (throughout time).

The aim is to develop a cointegration pairs trading strategy which identifies entry and exit points based on the spread between the pairs.
With that consideration in hand we should implement a mean-reversion approach to exploit arbitrage opportunities.

We can achieve that by calculating the Z-scores of the spreads as a metric to investigate how far the spread is from its mean. Then we can
either long or short depending on the z-score given that the spread is classified as a cointegrated series and we expect it to revert back
to its mean in the long run.

In [6]:
def trading_strategy(cointegration_results, data_dict, entry_z, exit_z, window_frame):
    
    # Every result is a cointegration pair with a specific beta
    # and its resulting spread is a series with entries the spread
    # at every time t. Each cointegration pair has a specific beta,
    # and hence a specific spread and means and stds.
    # Z scores are also series that have a singular value at every point
    # in time. 
    
    trades = []

    for result in cointegration_results:
        
        asset_1, asset_2, _, beta = result
        spread = data_dict[asset_1]['close'] - beta * data_dict[asset_2]['close']

        spread = spread.dropna().abs()

        # Use rolling mean and standard deviation
        rolling_mean = spread.rolling(window=window_frame).mean()
        rolling_std = spread.rolling(window=window_frame).std()
     
        z_scores = (spread - rolling_mean) / rolling_std

        
        entries = ((z_scores >entry_z) | (z_scores<-entry_z))
        exits =  ((z_scores<exit_z)&(z_scores>-exit_z))

        # Align signals with the spread index
        entries = entries.reindex(spread.index, fill_value=False)
        exits = exits.reindex(spread.index, fill_value=False)
        
        if entries.sum() == 0:

            print(f'No edge was found for pair {asset_1,asset_2}, do not trade it!')
        
        else:
            trades.append((asset_1,asset_2,spread,entries,exits))
        
    return trades

In [7]:

def backtest_strategy(trades):

    """
    Function that backtests the trading strategy developed
    above in vectorbt for a period of a month with minute resolution.
    """


    portfolios = []

    for asset_1, asset_2, spread, entries, exits in trades:
        try:
            portfolio = vbt.Portfolio.from_signals(
                close=spread,
                entries=entries,
                exits=exits,
                freq='1min'  # Assuming minute-level data
            )
            portfolios.append({
                'pair': (asset_1, asset_2),
                'portfolio': portfolio,
                'sharpe_ratio': portfolio.sharpe_ratio(),
                'total_return': portfolio.total_return(),
                'max_drawdown':portfolio.max_drawdown()
            })
        except Exception as e:
            print(f"Error creating portfolio for pair {asset_1, asset_2}: {e}")

    return portfolios


In [8]:
def optimize_strategy(cointegration_results, data_dict, window_frames, entry_zs, exit_zs):


    """
    Optimisation strategy that finds the optimal windows, entries and exits that yield
    the optimal total return.
    """
    
    best_results = []

    for result in cointegration_results:
        asset_1, asset_2, _, _ = result
        best_sharpe_ratio = -float('inf')
        best_total_return = 0
        best_params = None
        best_portfolio = None
        
        param_combinations = list(itertools.product(window_frames, entry_zs, exit_zs))
        
        for window_frame, entry_z, exit_z in param_combinations:
            
            
            trades = trading_strategy([result], data_dict, entry_z, exit_z, window_frame)
            portfolios = backtest_strategy(trades)

            for portfolio_result in portfolios:
                sharpe_ratio = portfolio_result['sharpe_ratio']
                total_return = portfolio_result['total_return']
                max_drawdown = portfolio_result['max_drawdown']
                
                if total_return > best_total_return:
                    best_total_return = total_return
                    best_params = (window_frame, entry_z, exit_z)
                    best_portfolio = portfolio_result['portfolio']
        
        if best_portfolio is not None:
            best_results.append({
                'pair': (asset_1, asset_2),
                'params': best_params,
                'portfolio': best_portfolio,
                'sharpe_ratio': best_sharpe_ratio
            })
    
    return best_results



In [9]:
# Define the hyperparameter grid that we want to optimise on.

window_frames = np.arange(10,80,10)
entry_zs = np.linspace(1.5,2.5,10)
exit_zs = np.linspace(0.5,1,10)

# Optimize the strategy.
results = optimize_strategy(EG_results, data_dict, window_frames, entry_zs, exit_zs)

# Print the stats for the best performing portfolio for every trading pair based on the returns.
for result in results:
    params = result['params']
    portfolio = result['portfolio']
    sharpe_ratio = result['sharpe_ratio']

    print(portfolio.stats())